## Required Installations

In [ ]:
!pip install gym[atari]

In [ ]:
!pip install opencv-python

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
mkdir: cannot create directory ‘rars’: File exists
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying 

In [ ]:
!pip install tensorboardX

In [ ]:
%cd /content/

/content


In [ ]:
import gym
import numpy as np
import cv2
import random
import torch
# import models
from torch import nn
from tensorboardX import SummaryWriter
import torchvision.utils as vutils
import os
import imageio
from torchsummary import summary

## [Code Base](https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On-Second-Edition/blob/master/Chapter03/03_atari_gan.py)

In [ ]:
LATENT_VECTOR_SIZE = 100
DISCR_FILTERS = 64
GENER_FILTERS = 64
BATCH_SIZE = 16

# dimension input image will be rescaled
IMAGE_SIZE = 64

LEARNING_RATE = 0.0001
REPORT_EVERY_ITER = 10000
SAVE_IMAGE_EVERY_ITER = 10000

In [ ]:
class InputWrapper(gym.ObservationWrapper):
  def __init__(self, *args):
    super(InputWrapper, self).__init__(*args)
    assert isinstance(self.observation_space, gym.spaces.Box)
    prev_space = self.observation_space
    self.observation_space = gym.spaces.Box(
        self.observation(prev_space.low),
        self.observation(prev_space.high),
        dtype=np.float32
    )

  def observation(self, obs):
    new_obs = cv2.resize(obs, (IMAGE_SIZE, IMAGE_SIZE))
    # transform (210, 160, 3) -> (3, 210, 160)
    new_obs = np.moveaxis(new_obs, 2, 0)
    return new_obs.astype(np.float32)

In [ ]:
def generate_samples(envs, batch_size=BATCH_SIZE):
  batch = [e.reset() for e in envs]
  env_gen = iter(lambda: random.choice(envs), None)

  while True:
    e = next(env_gen)
    obs, reward, is_done, _ = e.step(e.action_space.sample())
    if np.mean(obs) > 0.01:
      batch.append(obs)
    if len(batch) == batch_size:
      # Normalizing input between -1 to 1
      batch_np = np.array(batch, dtype=np.float32)
      batch_np *= 2.0/255.0 - 1.0
      yield torch.tensor(batch_np)
      batch.clear()
    if is_done:
      e.reset()

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()
        # this pipe converges image into the single number
        self.conv_pipe = nn.Sequential(
            nn.Conv2d(in_channels=input_shape[0], out_channels=DISCR_FILTERS,
                      kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS, out_channels=DISCR_FILTERS*2,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS*2),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 2, out_channels=DISCR_FILTERS * 4,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 4),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 4, out_channels=DISCR_FILTERS * 8,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(DISCR_FILTERS * 8),
            nn.ReLU(),
            nn.Conv2d(in_channels=DISCR_FILTERS * 8, out_channels=1,
                      kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        for layer in self.conv_pipe:
          x = layer(x)
          print(x.size())
        return x.view(-1, 1).squeeze(dim=1)


class Generator(nn.Module):
    def __init__(self, output_shape):
        super(Generator, self).__init__()
        # pipe deconvolves input vector into (3, 64, 64) image
        self.pipe = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=GENER_FILTERS * 8,
                               kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(GENER_FILTERS * 8),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 8, out_channels=GENER_FILTERS * 4,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 4),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 4, out_channels=GENER_FILTERS * 2,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS * 2),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS * 2, out_channels=GENER_FILTERS,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(GENER_FILTERS),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=GENER_FILTERS, out_channels=output_shape[0],
                               kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        # return self.pipe(x)
        for layer in self.pipe:
          x = layer(x)
          print(x.size())
        return x

In [ ]:
model = Discriminator(input_shape=(3, 64, 64))
print(model)
x = torch.randn(1, 3, 64, 64)
model(x)

Discriminator(
  (conv_pipe): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (12): Sigmoid()
  )
)
torch.Size([1, 64, 32, 32])
torch.Size([1, 64, 32, 32])
torch.Size([1, 128, 16, 16])
torch.Size([1, 128, 16, 16])
torch.Size([1, 128, 16, 16])
torch.Size([1, 256, 8, 8])
torch.Size([1, 256, 8, 8])
torch.Size([1, 256, 8, 8])
torch.Siz

tensor([0.5849], grad_fn=<SqueezeBackward1>)

In [ ]:
model = Generator(output_shape=(3, 64, 64))
print(model)
x = torch.randn(1, 100, 1, 1)
model(x)

Generator(
  (pipe): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)
torch.Size([1, 512, 4, 4])
torch.Size([1, 512, 4, 4])
torch.Size([1, 512, 4, 4])
torch.Size([1, 256, 8, 8])


tensor([[[[-0.4019, -0.3402, -0.3185,  ..., -0.4617, -0.1878, -0.1808],
          [ 0.1936,  0.5633, -0.2272,  ..., -0.2690, -0.0699,  0.0929],
          [ 0.0487,  0.3593,  0.1587,  ..., -0.0763, -0.5029, -0.1871],
          ...,
          [ 0.0438,  0.0740, -0.1021,  ...,  0.1895,  0.2004, -0.0818],
          [-0.1418, -0.3108, -0.1498,  ..., -0.2686, -0.2964,  0.1426],
          [-0.0679,  0.1171, -0.0870,  ..., -0.1997, -0.1087, -0.0620]],

         [[-0.1243, -0.2389, -0.0303,  ...,  0.0397,  0.0891,  0.0458],
          [-0.0809, -0.6348, -0.1133,  ..., -0.1643, -0.5549, -0.3384],
          [-0.1248, -0.2911,  0.7508,  ..., -0.4287, -0.3750, -0.0087],
          ...,
          [-0.1277, -0.5549,  0.2900,  ..., -0.3760, -0.2701, -0.2142],
          [-0.0039, -0.4392,  0.2489,  ..., -0.2774,  0.0533, -0.2155],
          [ 0.1072, -0.5004,  0.2728,  ..., -0.3679,  0.0271, -0.2887]],

         [[-0.0191,  0.3626, -0.1019,  ...,  0.1597, -0.1338, -0.1160],
          [ 0.0735,  0.7870,  

In [ ]:
device = torch.device("cuda")
envs = [InputWrapper(gym.make(game)) for game in ('Breakout-v0', 'MsPacman-v0', 'SpaceInvaders-v0')]
train_data = generate_samples(envs)
noise = torch.randn(BATCH_SIZE, 100, 1, 1)
input_shape = envs[0].observation_space.shape

In [ ]:
# training loops
net_discr = Discriminator(input_shape=input_shape).to(device)
net_gener = Generator(output_shape=input_shape).to(device)
writer = SummaryWriter()

objective = nn.BCELoss()

gen_optimizer = torch.optim.Adam(net_gener.parameters(), lr=0.0001, betas=(0.5, 0.999))
dis_optimizer = torch.optim.Adam(net_discr.parameters(), lr=0.0001, betas=(0.5, 0.999))

gen_losses = []
dis_losses = []
iter_no = 0

true_labels_v = torch.ones(BATCH_SIZE, device=device)
fake_labels_v = torch.zeros(BATCH_SIZE, device=device)

for batch_v in train_data:
  gen_input_v = torch.FloatTensor(
            BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1)
  gen_input_v.normal_(0, 1)
  gen_input_v = gen_input_v.to(device)
  batch_v = batch_v.to(device)
  gen_output_v = net_gener(gen_input_v)

  # train discriminator
  dis_optimizer.zero_grad()
  dis_output_true_v = net_discr(batch_v)
  dis_output_fake_v = net_discr(gen_output_v.detach())
  dis_loss = objective(dis_output_true_v, true_labels_v) + \
              objective(dis_output_fake_v, fake_labels_v)
  dis_loss.backward()
  dis_optimizer.step()
  dis_losses.append(dis_loss.item())

  # train generator
  gen_optimizer.zero_grad()
  dis_output_v = net_discr(gen_output_v)
  gen_loss_v = objective(dis_output_v, true_labels_v)
  gen_loss_v.backward()
  gen_optimizer.step()
  gen_losses.append(gen_loss_v.item())

  iter_no += 1
  if iter_no % REPORT_EVERY_ITER == 0:
    print(f"Iter {iter_no}: gen_loss={np.mean(gen_losses)}, dis_loss={np.mean(dis_losses)}")
    writer.add_scalar(
        "gen_loss", np.mean(gen_losses), iter_no)
    writer.add_scalar(
        "dis_loss", np.mean(dis_losses), iter_no)
    gen_losses = []
    dis_losses = []
    
    if not os.path.exists(f'{iter_no}'):
        os.makedirs(f'{iter_no}')
    torch.save(net_discr.state_dict(), f'{iter_no}/D.pkl')
    torch.save(net_gener.state_dict(), f'{iter_no}/G.pkl')

Iter 10000: gen_loss=11.015083949191167, dis_loss=0.0050717151021431395
Iter 20000: gen_loss=15.72122150363922, dis_loss=1.2060845846772138e-06
Iter 30000: gen_loss=20.840003999137878, dis_loss=4.699084856341073e-09
Iter 40000: gen_loss=24.618247546958923, dis_loss=6.929041242997869e-11
Iter 50000: gen_loss=26.428978769874572, dis_loss=1.4901162970204496e-12
Iter 60000: gen_loss=27.05434672050476, dis_loss=0.0
Iter 70000: gen_loss=26.78728736190796, dis_loss=1.043081425677883e-11
Iter 80000: gen_loss=26.567670729446412, dis_loss=5.215407128389415e-12
Iter 90000: gen_loss=26.977887541770937, dis_loss=0.0
Iter 100000: gen_loss=27.515558718681337, dis_loss=0.0
Iter 110000: gen_loss=27.650638795280457, dis_loss=4.470348713425665e-12
Iter 120000: gen_loss=27.69635641441345, dis_loss=0.0
Iter 130000: gen_loss=27.93418836631775, dis_loss=0.0
Iter 140000: gen_loss=26.922232485198975, dis_loss=9.685756197086447e-12
Iter 150000: gen_loss=27.55939024143219, dis_loss=0.0
